# Asymptotic Analysis

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cPickle as pickle
import json
from functools import partial
from multiprocessing import cpu_count, Pool
import pandas as pd
from itertools import product 
import string
from tqdm import tqdm_notebook
import copy
import time

In [3]:
from voting.voter import *
from voting.ranking import *
from voting.profile import *
from voting.voting_methods import *
from preflibtools.generate_profiles import *


## Preflib tools



In [4]:
def generate_names(num_cands, num_voters):

    cand_names = basic_cand_names = list(string.ascii_uppercase)

    voter_names = basic_voter_names = list(string.ascii_lowercase)

    while len(cand_names) < num_cands: 
        cand_names = cand_names + [''.join(_c) for _c in  list(product(cand_names, basic_cand_names))]

    while len(voter_names) < num_voters: 
        voter_names = voter_names + [''.join(_c) for _c in  list(product(voter_names, basic_voter_names))]
    
    return cand_names[0:num_cands], voter_names[0:num_voters]




In [5]:
def create_prof(num_voters, cmap):
    num_cands = len(cmap.keys())
    rmaps, rmapscounts = gen_impartial_culture_strict(num_voters, cmap)

    _prof = list()
    for _rmap in zip(rmapscounts,rmaps):

        _rank = list()
        for r in range(1,num_cands + 1):
            _rank.append(cmap[_rmap[1].keys()[_rmap[1].values().index(r)]])
        _prof += [tuple(_rank)] *_rmap[0]
        
    return _prof


In [6]:
def generate_candidate_maps(cand_names):
    
    num_cands = len(cand_names)
    cmap =  gen_cand_map(num_cands)
    for i,c in enumerate(cand_names):
        cmap[i+1] = c
    return cmap

In [7]:
voting_scenarios = pickle.load(open("./voting_data/voting_scenarios.pkl","rb"))

In [8]:
A="A"
B="B"
C="C"
D="D"

ws1 = [A, B, C]
ws2 = [A, B]

print [[c] for c in ws1]

[['A'], ['B'], ['C']]


In [9]:
def gen_winning_sets_from_all_tiebreakers(ws1, ws2):
    # to simplify the analysis, when the agent considers all 
    # tie breakers, each singleton from the winning set may be a winner
    
    ws1_ws = [[c] for c in ws1]
    ws2_ws = [[c] for c in ws2]
    
    return [_wss[0]  for _wss in list(product(ws1_ws, ws2_ws))], [_wss[1]  for _wss in list(product(ws1_ws, ws2_ws))]
    


In [10]:

#voting_sizes = [(3,4), (3,5), (3,6), (3,7), (3,10), (3,25), (3,40), (3,55), (3,70)]
voting_sizes = [(3,4)] #(3,4), (3,5), (3,6), (3,7), (3,10), (3,25), (3,40), (3,55), (3,70)]
prf_type = "profile" #"pointed_profile"  #"anecs" #pointed_anecs

#voting_scenarios = None

prob_type = "ic"

all_voting_methods = [borda, weak_nanson, strict_nanson, plurality] #, plurality]

voting_methods_sets = [(borda,), 
                       (borda, strict_nanson,), 
                       (borda, weak_nanson,), 
                       (borda, hare,), 
                       (borda, coombs,), 
                       (borda, plurality,)]

NUM_PROFILES = 10000 #2500
NUM_SIMULATIONS = 1

max_vms_set_size =  2


try: 
    voting_scenarios
    print "Using all profiles when available...." if not None else "Not reading the saved voting scenarios"
except: 
    voting_scenarios = None

output_single_vm = {vsize: {vm.__doc__: {"With tie breaker":list(),
                                         "Weak dominance": list(),
                                         "Optimist": list(),
                                         "Pessimist": list(),
                                         "Sure manipulation, uncertainty about tie breaker": list(),
                                         "Safe manipulation, uncertainty about tie breaker": list()} 
                            for vm in all_voting_methods} for vsize in voting_sizes}

output_many_vms = {vsize: {vms_to_str(vms): {"Surely weak dom. manip.": list(),
                                             "Safely weak dom. manip.": list()} 
                   for vms in voting_methods_sets} for vsize in voting_sizes}

def apply_tie_breaker(vm, tie_breaker):
    return lambda prf: list(tie_breaker(vm(prf)))

def find_manipulation_instances_for_profile(vm, other_voting_methods, 
                                            cand_names, voter_names, 
                                            _prof):
    
    tie_breakers = create_all_tie_breakers(cand_names)
    
    output = {"With tie breaker":False, 
              "Weak dominance": False,
              "Optimist": False,
              "Pessimist": False,
              "Sure manipulation, uncertainty about tie breaker": False,
              "Safe manipulation, uncertainty about tie breaker": False}
    
    for voter_name in voter_names:
        
        pprof = create_pointed_profile(voter_names, _prof, voter_name)
       
        voter = pprof.voter
    
        vm_with_tie_breaker = apply_tie_breaker(vm, tie_breakers[0])

        #vm_all_tie_breakers = [apply_tie_breaker(vm, tb) for tb in tie_breakers]
    
        for new_ranking in pprof.all_rankings:

            updated_pprof = pprof.create_new_profile(new_ranking)

            if voter.sure_dominance(vm_with_tie_breaker(updated_pprof), vm_with_tie_breaker(pprof), voter.weak_dom):
                output["With tie breaker"] = True

            if voter.sure_dominance([vm(updated_pprof)], [vm(pprof)], voter.weak_dom):
                output["Weak dominance"] = True

            if voter.sure_dominance([vm(updated_pprof)], [vm(pprof)], voter.optimist_prefers):
                output["Optimist"] = True


            if voter.sure_dominance([vm(updated_pprof)], [vm(pprof)], voter.pessimist_prefers):
                output["Pessimist"] = True

            ws_from_all_tiebreakers = gen_winning_sets_from_all_tiebreakers(vm(updated_pprof), vm(pprof))
            if voter.sure_dominance(ws_from_all_tiebreakers[0], 
                                    ws_from_all_tiebreakers[1], 
                                    voter.weak_dom):
                output["Sure manipulation, uncertainty about tie breaker"] = True

            if voter.safe_dominance(ws_from_all_tiebreakers[0], 
                                    ws_from_all_tiebreakers[1], 
                                    voter.AAdom, voter.weak_dom):
                output["Safe manipulation, uncertainty about tie breaker"] = True


    return output


def find_manipulation_instances_for_vm_sets_for_profile(vms, 
                                                        cand_names, 
                                                        voter_names,
                                                        _prof):
        
    output = {"Surely weak dom. manip. for {}".format(vms_to_str(vms)): False,
              "Safely weak dom. manip. for {}".format(vms_to_str(vms)): False} 

    for voter_name in voter_names:
        
        pprof = create_pointed_profile(voter_names, _prof, voter_name)
       
        voter = pprof.voter
        
        winning_sets = [vm(pprof) for vm in vms]
        for new_ranking in pprof.all_rankings:

            updated_pprof = pprof.create_new_profile(new_ranking)
            
            updated_winning_sets = [vm(updated_pprof) for vm in vms]
            if voter.sure_dominance(updated_winning_sets, 
                                    winning_sets, 
                                    voter.weak_dom):
                output["Surely weak dom. manip. for {}".format(vms_to_str(vms))] = True

            if voter.safe_dominance(updated_winning_sets, 
                                    winning_sets, 
                                    voter.AAdom,
                                    voter.weak_dom):
                output["Safely weak dom. manip. for {}".format(vms_to_str(vms))] = True

    return output


def find_manipulation_instances_for_vm_sets_for_profile2(all_vms_sets, 
                                                        cand_names, 
                                                        voter_names,
                                                         all_rankings,
                                                        _prof):
        
    output = {vms_to_str(vms): {"Surely weak dom. manip.": False,
                                "Safely weak dom. manip.": False} 
              for vms in all_vms_sets} 

    for voter_name in voter_names:
        
        pprof = create_pointed_profile(voter_names, _prof, voter_name)
       
        voter = pprof.voter
        
        winning_sets = {vms_to_str(vms): [vm(pprof) for vm in vms] for vms in all_vms_sets}
        for new_ranking in all_rankings:

            updated_pprof = pprof.create_new_profile(new_ranking)
            
            updated_winning_sets = {vms_to_str(vms): [vm(updated_pprof) for vm in vms] for vms in all_vms_sets}
            
            for vms in all_vms_sets:
                if voter.sure_dominance(updated_winning_sets[vms_to_str(vms)], 
                                        winning_sets[vms_to_str(vms)], 
                                        voter.weak_dom):
                    output[vms_to_str(vms)]["Surely weak dom. manip."] = True

                if voter.safe_dominance(updated_winning_sets[vms_to_str(vms)], 
                                        winning_sets[vms_to_str(vms)],
                                        voter.AAdom,
                                        voter.weak_dom):
                    output[vms_to_str(vms)]["Safely weak dom. manip."] = True

    return output


def find_manipulation_instances_for_vm_sets_for_profile_for_all_vm_sets(all_vms_sets, 
                                                                        cand_names,
                                                                        voter_names,
                                                                        _prof):
    
    output = {vms_to_str(vms): dict() for vms in all_vms_sets}
    
    for vms in all_vms_sets:
        output[vms_to_str(vms)] = find_manipulation_instances_for_vm_sets_for_profile(vms, 
                                                                                      cand_names, 
                                                                                      voter_names,
                                                                                      _prof)
    return output

def generate_profiles_ic(num_cands, num_voters, num_profiles, voting_scenarios, is_pointed=False):
        
    cand_names, voter_names = generate_names(num_cands, num_voters)

    cmap = generate_candidate_maps(cand_names)

    num_cands = len(cand_names)
    num_voters = len(voter_names)
    
    profiles = list()
        
    _vs =  filter(lambda vs: vs["num_candidates"] == num_cands and vs["num_voters"] == num_voters, 
                  voting_scenarios) if voting_scenarios is not None else list()

    if len(_vs) == 1:
        vs = _vs[0] 
        profiles = vs["pointed_profiles"] if is_pointed else vs["profiles"]
        cand_names = vs["candidates"]
        voter_names = vs["voter_names"]
    
    else: # need to generate them
        
        cmap = generate_candidate_maps(cand_names)

        if is_pointed: # generate pointed profiles
            
            num_pointed_profiles = num_profiles * len(voter_names)
            
            profiles = [(create_prof(num_voters, cmap),
                         random.choice(voter_names)) for _ in range(0, num_pointed_profiles)]

        else: # generate profiles
            
            profiles = [create_prof(num_voters, cmap) for _ in range(0, num_profiles)]

    return profiles, cand_names, voter_names


prf_types = {
    "profile": {"is_pointed": False, 
                "find_manip_instances": find_manipulation_instances_for_profile,
                "find_manip_instances_vm_sets": find_manipulation_instances_for_vm_sets_for_profile,
                "generate_profiles": generate_profiles_ic
               },
}


for vsize in tqdm_notebook(voting_sizes, desc="Voting Sizes"):
    print vsize
    t0 = time.time()
    num_cands  = vsize[0]
    num_voters = vsize[1]
    
    
    have_all_profiles =  voting_scenarios is not None and len(filter(lambda vs: vs["num_candidates"] == num_cands and vs["num_voters"] == num_voters, voting_scenarios)) == 1
    num_simulations = NUM_SIMULATIONS if not have_all_profiles else 1
    
    print "\nRunning {} simulations".format(num_simulations)

    for sim in tqdm_notebook(range(0, num_simulations),leave=False, desc="Sims"): 
        
        print "sim {}".format(sim)        
        print "\nGenerate {} ...\n".format(prf_type)
        generate_profs = prf_types[prf_type]["generate_profiles"]
        find_manip_instances = prf_types[prf_type]["find_manip_instances"]
        find_manip_instances_for_vm_sets = prf_types[prf_type]["find_manip_instances_vm_sets"]

        profs, cand_names, voter_names = generate_profs(num_cands, 
                                                        num_voters,
                                                        NUM_PROFILES,
                                                            voting_scenarios,
                                                        is_pointed = prf_types[prf_type]["is_pointed"])
    
        #tie_breakers = create_all_tie_breakers(cand_names)
        all_rankings = generate_linear_orderings(cand_names)
        num_profs = len(profs)
                    
        print "number for profiles is {}".format(num_profs)
        prof_to_manip_instance = partial(find_manipulation_instances_for_vm_sets_for_profile2,
                                         voting_methods_sets,
                                         cand_names,
                                         voter_names,
                                         all_rankings)
            
            
        chunksize = int(0.05 * num_profs) if int(0.05 * num_profs) < 10 else 10
        print "chunksize is {}".format(chunksize)
        p = Pool(None)
        print "Using {} CPUs".format(cpu_count())            
          
        instances = list()
        for _ in tqdm_notebook(p.imap_unordered(prof_to_manip_instance, profs), total=num_profs, leave=False):
            instances.append(_)

        #pool.imap_unordered()
        #instances = p.map(prof_to_manip_instance, profs)
            
        p.close()
        p.join()
        
        print "now calculating output for the vms_sets...."
        #print "vsize is {}".format(vsize)

        for vms in voting_methods_sets:
            output_many_vms[vsize][vms_to_str(vms)]["Surely weak dom. manip."].append(float(sum([i[vms_to_str(vms)]["Surely weak dom. manip."] for i in instances])) / float(num_profs))
            output_many_vms[vsize][vms_to_str(vms)]["Safely weak dom. manip."].append(float(sum([i[vms_to_str(vms)]["Safely weak dom. manip."] for i in instances])) / float(num_profs))

            print "output_many_vms "
            #print output_many_vms
            print 
            
        #print output_many_vms
        pickle.dump(output_many_vms, open("./output_many_vms_percentage_manip_{}_2.pkl".format(prob_type + "_" + prf_type),"w"))
        t1 = time.time()

        print "execution time: {}".format(t1-t0)
        
print "Done."

Using all profiles when available....


(5, 5)

Running 1 simulations


sim 0

Generate profile ...

number for profiles is 10000
chunksize is 10
Using 8 CPUs


now calculating output for the vms_sets....
output_many_vms 

output_many_vms 

output_many_vms 

output_many_vms 

output_many_vms 

output_many_vms 

execution time: 7350.97273397
(5, 6)

Running 1 simulations


sim 0

Generate profile ...

number for profiles is 10000
chunksize is 10
Using 8 CPUs


now calculating output for the vms_sets....
output_many_vms 

output_many_vms 

output_many_vms 

output_many_vms 

output_many_vms 

output_many_vms 

execution time: 11126.054004

Done.
